In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
import re
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
shot = 115
picL = 18333
picR = 18158
indexL = 2296
indexR = 641
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
#shot115,235
F_10point_2 = [[  5.59273094e-07,  -6.08650734e-06,   2.05586754e-03], [  7.17519639e-06,   1.44074201e-06,  -7.14172969e-03],[ -3.18532599e-03,   4.59309570e-03,   9.99956762e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

In [2]:
tL = pd.read_csv('./Data/Shot{}/trackL3_frame_inv.csv'.format(shot))
tL.head()
#tL['particle'].unique()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.400000,0.0,0,13.825319,4.0,26.0,3.950091,0.845154,885.000000,240.000000,157.018865
1,0,NaN,NaN,NaN,NaN,NaN,0.395833,0.0,0,31.600729,6.0,68.0,7.900182,0.866025,601.101562,291.992188,35.599383
2,0,NaN,NaN,NaN,NaN,NaN,0.359477,0.0,0,53.079350,12.0,126.0,15.306603,0.843580,766.948837,368.120930,40.474716
3,0,NaN,NaN,NaN,NaN,NaN,0.437086,0.0,0,52.832469,15.0,129.0,15.553484,0.840004,732.070093,393.046729,38.276166
4,0,NaN,NaN,NaN,NaN,NaN,0.454545,0.0,0,46.413571,16.0,99.0,13.825319,0.837931,783.968085,401.000000,47.366911


In [3]:
tR = pd.read_csv('./Data/Shot{}/trackR3_frame_inv.csv'.format(shot))
tR.head()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.360000,0.0,0,18.950972,0.0,43.0,5.016434,0.857493,780.102941,214.897059,174.020553
1,0,NaN,NaN,NaN,NaN,NaN,0.379310,0.0,0,32.606819,2.0,74.0,8.360723,0.862316,555.991453,292.948718,46.182622
2,0,NaN,NaN,NaN,NaN,NaN,0.288660,0.0,0,35.672417,3.0,75.0,8.639413,0.870524,549.039062,295.921875,41.276336
3,0,NaN,NaN,NaN,NaN,NaN,0.414013,0.0,0,60.475894,4.0,149.0,16.721445,0.850589,581.170507,367.119816,38.744901
4,0,NaN,NaN,NaN,NaN,NaN,0.426471,0.0,0,52.393862,8.0,124.0,14.491919,0.850532,688.835106,393.909574,21.550071


In [9]:
ipL0 = indexL
TrackL = tL[tL['particle']==ipL0]
lenL = len(TrackL)
ipR0 = indexR
TrackR = tR[tR['particle']==ipR0] 
lenR = len(TrackR)
print(ipL0,lenL,ipR0,lenR)
plt.figure(figsize = (24,24))  # make a new figure
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax2.imshow(v0L+bk0L)
ax2.scatter(TrackL['x'],TrackL['x'],color='r',marker=".",s=1)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax4.imshow(v0R+bk0R)
ax4.scatter(TrackR['x'],TrackR['y'],color='y',marker=".",s=1)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax1.scatter(TrackL['frame'],TrackL['mass'],color='r',marker=".",s=1)
ax1.scatter(TrackR['frame'],TrackR['mass'],color='y',marker=".",s=1)
ax1.set_xlabel('Frame')
ax1.set_ylabel('Mass')
plt.savefig('./Data/Shot{}/Pair_{}_{}.pdf'.format(shot,ipL0,ipR0))
PairL=[]
PairR=[]
textL= open("calL_{}_{}.csv".format(shot,ipL0), "w")
textR= open("calR_{}_{}.csv".format(shot,ipR0), "w")
for iframeL in TrackL['frame'].unique():
    for iframeR in TrackR['frame'].unique():
        if(iframeL == iframeR):
            TrackL1=TrackL[TrackL['frame']==iframeL]
            TrackR1=TrackR[TrackR['frame']==iframeR]            
            ixL = TrackL1['x'].iloc[0]
            iyL = TrackL1['y'].iloc[0]  
            ixR = TrackR1['x'].iloc[0]
            iyR = TrackR1['y'].iloc[0]
            textL.write("{},{}\n".format(ixL,iyL))
            textR.write("{},{}\n".format(ixR,iyR))
textL.close()
textR.close()
'''
            print(iframeL,ixL,iyL,ixR,iyR)
            templistL = [ixL,iyL]
            templistR = [ixR,iyR]
            PairL.append(templistL)
            PairR.append(templistR)
PL = pd.DataFrame(PairL)
PR = pd.DataFrame(PairR)
PL.to_csv("calL_{}_{}.csv".format(shot,ipL0))
PR.to_csv("calR_{}_{}.csv".format(shot,ipR0))
'''

(2296, 1128, 641, 1034)


'\n            print(iframeL,ixL,iyL,ixR,iyR)\n            templistL = [ixL,iyL]\n            templistR = [ixR,iyR]\n            PairL.append(templistL)\n            PairR.append(templistR)\nPL = pd.DataFrame(PairL)\nPR = pd.DataFrame(PairR)\nPL.to_csv("calL_{}_{}.csv".format(shot,ipL0))\nPR.to_csv("calR_{}_{}.csv".format(shot,ipR0))\n'